In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.1


In [2]:
!nvidia-smi

Tue Apr 23 01:31:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P0              29W / 115W |     11MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import doxapy
from tabulate import tabulate

def evaluate(gt_imgs, pr_imgs, print_results=True):
    perf_data = []

    for gt_img, pr_img in zip(gt_imgs, pr_imgs):
        perf_data.append(doxapy.calculate_performance(gt_img, pr_img))

    mean_values = {}

    for d in perf_data:
        for key, value in d.items():
            if key not in mean_values:
                mean_values[key] = []
            mean_values[key].append(value)
        
    nice_names = {
        'accuracy': 'Accuracy (%)',
        'fm': 'F-measure',
        'mcc': 'Matthews Correlation Coefficient',
        'psnr': 'Peak Signal-to-Noise Ratio (PSNR)',
        'nrm': 'Normalized Root Mean Square Error (NRM)',
        'drdm': 'Distance-based Performance Measure (DRDM)'
    }

    mean_values_single = {nice_names[key]: round(value[0], 2) for key, value in mean_values.items()}
    
    if(print_results == True):
        data = [
            ["Metric", "Processed Images"],
            ["Accuracy (%)", "{:.2f}".format(mean_values_single['accuracy'])],
            ["F-measure", "{:.2f}".format(mean_values_single['fm'])],
            ["Matthews Correlation Coefficient", "{:.2f}".format(mean_values_single['mcc'])],
            ["Peak Signal-to-Noise Ratio (PSNR)", "{:.2f}".format(mean_values_single['psnr'])],
            ["Normalized Root Mean Square Error (NRM)", "{:.2f}".format(mean_values_single['nrm'])],
            ["Distance-based Performance Measure (DRDM)", "{:.2f}".format(mean_values_single['drdm'])],
        ]

        print(tabulate(data, headers="firstrow", tablefmt="grid"))

    return mean_values_single

def show_metrics_table(metrics_dicts):
    metrics_names = list(metrics_dicts.values())[0].keys()

    headers = ["Metric"] + list(metrics_dicts.keys())

    table_data = []

    for metric_name in metrics_names:
        row_data = [metric_name]
        for metrics_dict in metrics_dicts.values():
            row_data.append("{:.2f}".format(metrics_dict[metric_name]))
        table_data.append(row_data)

    print(tabulate(table_data, headers=headers, tablefmt="grid"))

In [5]:
from tabulate import tabulate

def evaluate(gt_imgs, pr_imgs, print_results=True):
    perf_data = []

    # Calculate performance for each ground truth and predicted image pair
    for gt_img, pr_img in zip(gt_imgs, pr_imgs):
        perf_data.append(doxapy.calculate_performance(gt_img, pr_img))

    mean_values = {}

    # Aggregate values for each metric across all image comparisons
    for d in perf_data:
        for key, value in d.items():
            if key not in mean_values:
                mean_values[key] = []
            mean_values[key].append(value)
        
    # Prepare nicer names for the metrics to display
    nice_names = {
        'accuracy': 'Accuracy (%)',
        'fm': 'F-measure',
        'mcc': 'Matthews Correlation Coefficient',
        'psnr': 'Peak Signal-to-Noise Ratio (PSNR)',
        'nrm': 'Normalized Root Mean Square Error (NRM)',
        'drdm': 'Distance-based Performance Measure (DRDM)'
    }

    # Prepare a dictionary with the average values for each metric, if present
    mean_values_single = {}
    for key, values in mean_values.items():
        if key in nice_names:
            mean_values_single[nice_names[key]] = round(sum(values) / len(values), 2)

    # Print results if specified
    if print_results:
        data = [["Metric", "Processed Images"]]
        for metric_name, metric_value in mean_values_single.items():
            data.append([metric_name, "{:.2f}".format(metric_value)])

        print(tabulate(data, headers="firstrow", tablefmt="grid"))

    return mean_values_single

In [8]:
import cv2
# from assignment_1.binarization.evaluation import evaluate
img1 = 'data/out/dibco1.png'
img2 = 'data/out/dibco2.png'
img3 = 'data/out/dibco3.png'
img4 = 'data/out/dibco4.png'
img5 = 'data/out/dibco5.png'

gt1 = "data/davu2024/DIBC02009/dibco_img0001_gt.tif"
gt2 = "data/davu2024/DIBC02009/dibco_img0002_gt.tif"
gt3 = "data/davu2024/DIBC02009/dibco_img0003_gt.tif"
gt4 = "data/davu2024/DIBC02009/dibco_img0004_gt.tif"
gt5 = "data/davu2024/DIBC02009/dibco_img0005_gt.tif"

gt_imgs = [cv2.imread(gt1, cv2.IMREAD_GRAYSCALE), cv2.imread(gt2, cv2.IMREAD_GRAYSCALE), cv2.imread(gt3, cv2.IMREAD_GRAYSCALE), cv2.imread(gt4, cv2.IMREAD_GRAYSCALE), cv2.imread(gt5, cv2.IMREAD_GRAYSCALE)]

pr_imgs = [cv2.imread(img1, cv2.IMREAD_GRAYSCALE), cv2.imread(img2, cv2.IMREAD_GRAYSCALE), cv2.imread(img3, cv2.IMREAD_GRAYSCALE), cv2.imread(img4, cv2.IMREAD_GRAYSCALE), cv2.imread(img5, cv2.IMREAD_GRAYSCALE)]

evaluate(gt_imgs, pr_imgs, print_results=True)

+-------------------------------------------+--------------------+
| Metric                                    |   Processed Images |
+===========================================+====================+
| Accuracy (%)                              |              96.78 |
+-------------------------------------------+--------------------+
| F-measure                                 |              62.9  |
+-------------------------------------------+--------------------+
| Matthews Correlation Coefficient          |               0.66 |
+-------------------------------------------+--------------------+
| Peak Signal-to-Noise Ratio (PSNR)         |              15.83 |
+-------------------------------------------+--------------------+
| Normalized Root Mean Square Error (NRM)   |               0.24 |
+-------------------------------------------+--------------------+
| Distance-based Performance Measure (DRDM) |               9.94 |
+-------------------------------------------+-----------------

{'Accuracy (%)': 96.78,
 'F-measure': 62.9,
 'Matthews Correlation Coefficient': 0.66,
 'Peak Signal-to-Noise Ratio (PSNR)': 15.83,
 'Normalized Root Mean Square Error (NRM)': 0.24,
 'Distance-based Performance Measure (DRDM)': 9.94}